In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import gc

In [ ]:
# csv파일을 parquet파일로 변환하기 위한 함수

def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet('./' + save_name + '.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [ ]:
# 파일 형식 변경

csv_to_parquet('./train.csv', 'train')
csv_to_parquet('./test.csv', 'test')

In [ ]:
# 파일 로드
# test파일은 target이 아에 없으므로 train에서 나눠서 사용

train = pd.read_parquet('./train.parquet')
# test = pd.read_parquet('./test.parquet')

In [ ]:
# 데이터 전처리
# [요일, 시작 지점의 회전제한 유무, 도착지점의 회전제한 유무]
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    print(i, le.classes_)

In [ ]:
# 주말인지 아닌지 판별을 위한 컬럼 추가
train['weekend']=0
train.loc[train['day_of_week'] == 4,'weekend'] = 1
train.loc[train['day_of_week'] == 5,'weekend'] = 1

In [ ]:
# 월 컬럼 추가
train['month']=train['base_date']
train['month'] = train['month'].apply(lambda x: int(str(x)[4:6]))

In [ ]:
train['weather']=0


#계절별 분류 month를
train.loc[train['month']==12,'weather']=1
train.loc[train['month']==1,'weather']=1
train.loc[train['month']==2,'weather']=1

train.loc[train['month']==3,'weather']=2
train.loc[train['month']==4,'weather']=2
train.loc[train['month']==5,'weather']=2

train.loc[train['month']==6,'weather']=3
train.loc[train['month']==7,'weather']=3
train.loc[train['month']==8,'weather']=3

train.loc[train['month']==9,'weather']=4
train.loc[train['month']==10,'weather']=4
train.loc[train['month']==11,'weather']=4

# 3~5 봄(2) 6~8 여름(3) 9~11 가을(4) 12~2 겨울(1)  법적 계절 분류

In [ ]:
# 4시간씩 잘라봄 (base_hour) 기준

In [ ]:
train['base_time']=0

In [ ]:
train.loc[(train['base_hour']  >= 0)&(train['base_hour'] <= 4), 'base_time']=0

train.loc[(train['base_hour']  >= 5)&(train['base_hour'] <= 9), 'base_time']=1

train.loc[(train['base_hour']  >= 10)&(train['base_hour'] <= 14), 'base_time']=2

train.loc[(train['base_hour']  >= 15)&(train['base_hour'] <= 19), 'base_time']=3

train.loc[train['base_hour']  >= 20,'base_time']=4

In [ ]:
# 정리한 데이터프레임 저장

train.to_parquet('./train_pwm.parquet')
print('Done.')

---

# 위도 경도 삭제 (2번째 시도)

In [ ]:
# train2  deeopcopy 및 위도 경도 삭제 

In [ ]:
import copy

In [ ]:
train2=copy.deepcopy(train)

In [ ]:
train2=train2.drop(['end_latitude','end_longitude','start_latitude','start_longitude'],axis=1)

In [ ]:
# 정리한 데이터프레임 저장

train2.to_parquet('./train2_pwm.parquet')
print('Done.')

In [ ]:
train2 = pd.read_parquet('./train2_pwm.parquet')